# Keyword Embedding & HDBSCAN Topic Clustering

This notebook:
1. Embeds speech keywords using Turkish sentence-transformers model
2. Clusters speeches using HDBSCAN algorithm
3. Generates topic labels based on representative keywords
4. Uploads new topic assignments to Elasticsearch
5. Visualizes clusters with UMAP

**Model**: `trmteb/turkish-embedding-model-fine-tuned` (768-dimensional embeddings)

**HDBSCAN**: Hierarchical density-based clustering (no need to specify number of clusters)

## 1. Setup and Imports

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from typing import List, Dict, Tuple

# Sentence transformers for embedding
from sentence_transformers import SentenceTransformer

# Clustering
import hdbscan
from sklearn.metrics import silhouette_score

# Dimensionality reduction for visualization
import umap

# Elasticsearch
from elasticsearch import Elasticsearch, helpers
from tqdm.auto import tqdm

# Configuration
KEYWORDS_CSV = "../data/speech_keywords.csv"
EMBEDDINGS_FILE = "../data/keyword_embeddings.npy"
OUTPUT_CSV = "../data/speech_keywords_with_topics.csv"
MODEL_NAME = "trmteb/turkish-embedding-model-fine-tuned"
ELASTICSEARCH_HOST = os.getenv("ELASTICSEARCH_HOST", "http://localhost:9200")
ELASTICSEARCH_INDEX = os.getenv("ELASTICSEARCH_INDEX", "parliament_speeches")

print("✅ Imports complete")

ModuleNotFoundError: No module named 'seaborn'

## 2. Load Keywords Data

In [16]:
# Load the CSV with keywords
print(f"📥 Loading keywords from {KEYWORDS_CSV}...")
df = pd.read_csv(KEYWORDS_CSV)

print(f"\n📊 Dataset info:")
print(f"   Total speeches: {len(df):,}")
print(f"   Columns: {df.columns.tolist()}")

# Display sample
print(f"\n📋 Sample data:")
df.head()

NameError: name 'KEYWORDS_CSV' is not defined

## 3. Load Turkish Embedding Model

In [ ]:
print(f"🔄 Loading model: {MODEL_NAME}...")
print("   This may take a few minutes on first run...\n")

# Load the model
model = SentenceTransformer(MODEL_NAME)

# Check model details
embedding_dim = model.get_sentence_embedding_dimension()
print(f"✅ Model loaded successfully!")
print(f"   Embedding dimension: {embedding_dim}")
print(f"   Max sequence length: {model.max_seq_length}")

## 4. Generate Keyword Embeddings

In [ ]:
def generate_embeddings(keywords_list: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Generate embeddings for a list of keyword strings.
    
    Args:
        keywords_list: List of comma-separated keyword strings
        batch_size: Batch size for encoding
    
    Returns:
        numpy array of shape (n_samples, embedding_dim)
    """
    print(f"\n🔄 Generating embeddings for {len(keywords_list):,} speeches...")
    print(f"   Batch size: {batch_size}")
    
    # Encode in batches with progress bar
    embeddings = model.encode(
        keywords_list,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    print(f"\n✅ Embeddings generated!")
    print(f"   Shape: {embeddings.shape}")
    print(f"   Memory: {embeddings.nbytes / 1e6:.2f} MB")
    
    return embeddings

# Check if embeddings already exist
if os.path.exists(EMBEDDINGS_FILE):
    print(f"\n📂 Found existing embeddings at {EMBEDDINGS_FILE}")
    response = input("Load existing embeddings? (y/n): ")
    
    if response.lower() == 'y':
        embeddings = np.load(EMBEDDINGS_FILE)
        print(f"✅ Loaded embeddings: {embeddings.shape}")
    else:
        embeddings = generate_embeddings(df['keywords'].tolist())
        np.save(EMBEDDINGS_FILE, embeddings)
        print(f"💾 Saved embeddings to {EMBEDDINGS_FILE}")
else:
    embeddings = generate_embeddings(df['keywords'].tolist())
    np.save(EMBEDDINGS_FILE, embeddings)
    print(f"💾 Saved embeddings to {EMBEDDINGS_FILE}")

## 5. HDBSCAN Clustering

In [ ]:
def cluster_with_hdbscan(
    embeddings: np.ndarray,
    min_cluster_size: int = 50,
    min_samples: int = 10,
    metric: str = 'euclidean',
    cluster_selection_epsilon: float = 0.0
) -> Tuple[np.ndarray, hdbscan.HDBSCAN]:
    """
    Cluster embeddings using HDBSCAN.
    
    Args:
        embeddings: numpy array of embeddings
        min_cluster_size: minimum size of clusters
        min_samples: number of samples in a neighborhood
        metric: distance metric to use
        cluster_selection_epsilon: distance threshold for merging clusters
    
    Returns:
        Tuple of (cluster_labels, clusterer_object)
    """
    print(f"\n🔄 Running HDBSCAN clustering...")
    print(f"   Parameters:")
    print(f"   - min_cluster_size: {min_cluster_size}")
    print(f"   - min_samples: {min_samples}")
    print(f"   - metric: {metric}")
    print(f"   - cluster_selection_epsilon: {cluster_selection_epsilon}")
    
    # Create and fit clusterer
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric=metric,
        cluster_selection_epsilon=cluster_selection_epsilon,
        prediction_data=True
    )
    
    labels = clusterer.fit_predict(embeddings)
    
    # Statistics
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_outliers = np.sum(labels == -1)
    
    print(f"\n✅ Clustering complete!")
    print(f"   Number of clusters: {n_clusters}")
    print(f"   Outliers: {n_outliers:,} ({n_outliers/len(labels)*100:.1f}%)")
    print(f"   Clustered speeches: {len(labels) - n_outliers:,}")
    
    return labels, clusterer

# Run clustering with default parameters
# Adjust these parameters based on your data
cluster_labels, clusterer = cluster_with_hdbscan(
    embeddings,
    min_cluster_size=50,  # Minimum 50 speeches per cluster
    min_samples=15,       # At least 15 neighbors to form core point
    metric='euclidean'    # Euclidean distance in embedding space
)

# Add to dataframe
df['hdbscan_topic_id'] = cluster_labels

## 6. Cluster Statistics & Analysis

In [ ]:
# Cluster size distribution
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()

print("📊 Cluster Size Distribution:\n")
for cluster_id, count in cluster_counts.items():
    if cluster_id == -1:
        print(f"   Cluster {cluster_id:3d} (Outliers): {count:6,} speeches")
    else:
        print(f"   Cluster {cluster_id:3d}: {count:6,} speeches")

# Plot cluster distribution
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
cluster_counts_no_outliers = cluster_counts[cluster_counts.index != -1]
plt.bar(range(len(cluster_counts_no_outliers)), cluster_counts_no_outliers.values)
plt.xlabel('Cluster ID')
plt.ylabel('Number of Speeches')
plt.title('Cluster Size Distribution (excluding outliers)')
plt.xticks(range(0, len(cluster_counts_no_outliers), max(1, len(cluster_counts_no_outliers)//20)))

plt.subplot(1, 2, 2)
plt.hist(cluster_counts_no_outliers.values, bins=30, edgecolor='black')
plt.xlabel('Cluster Size')
plt.ylabel('Frequency')
plt.title('Distribution of Cluster Sizes')

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n📈 Summary Statistics (excluding outliers):")
print(f"   Mean cluster size: {cluster_counts_no_outliers.mean():.1f}")
print(f"   Median cluster size: {cluster_counts_no_outliers.median():.1f}")
print(f"   Largest cluster: {cluster_counts_no_outliers.max():,} speeches")
print(f"   Smallest cluster: {cluster_counts_no_outliers.min():,} speeches")

## 7. Generate Topic Labels

In [ ]:
def generate_topic_labels(df: pd.DataFrame, n_keywords: int = 5) -> Dict[int, str]:
    """
    Generate topic labels based on most common keywords in each cluster.
    
    Args:
        df: DataFrame with 'hdbscan_topic_id' and 'keywords' columns
        n_keywords: Number of top keywords to use for label
    
    Returns:
        Dictionary mapping cluster_id to topic label
    """
    topic_labels = {}
    
    for cluster_id in sorted(df['hdbscan_topic_id'].unique()):
        # Get all speeches in this cluster
        cluster_speeches = df[df['hdbscan_topic_id'] == cluster_id]
        
        # Extract all keywords from this cluster
        all_keywords = []
        for keywords_str in cluster_speeches['keywords']:
            if pd.notna(keywords_str):
                keywords = [k.strip() for k in str(keywords_str).split(',')]
                all_keywords.extend(keywords)
        
        # Count keyword frequencies
        keyword_counts = Counter(all_keywords)
        
        # Get top N keywords
        top_keywords = [kw for kw, count in keyword_counts.most_common(n_keywords)]
        
        # Create label
        if cluster_id == -1:
            topic_labels[cluster_id] = "Outliers"
        else:
            topic_labels[cluster_id] = ", ".join(top_keywords)
    
    return topic_labels

print("🏷️  Generating topic labels...\n")
topic_labels = generate_topic_labels(df, n_keywords=5)

# Add labels to dataframe
df['hdbscan_topic_label'] = df['hdbscan_topic_id'].map(topic_labels)

# Display topics
print("📋 Generated Topics:\n")
for cluster_id, label in sorted(topic_labels.items()):
    count = len(df[df['hdbscan_topic_id'] == cluster_id])
    if cluster_id == -1:
        print(f"   Topic {cluster_id:3d} ({count:6,} speeches): {label}")
    else:
        print(f"   Topic {cluster_id:3d} ({count:6,} speeches): {label[:80]}..." if len(label) > 80 else f"   Topic {cluster_id:3d} ({count:6,} speeches): {label}")

## 8. Visualize Clusters with UMAP

In [ ]:
print("🔄 Reducing dimensions with UMAP for visualization...")
print("   This may take a few minutes...\n")

# Reduce to 2D for visualization
reducer = umap.UMAP(
    n_neighbors=15,
    n_components=2,
    metric='euclidean',
    random_state=42
)

embeddings_2d = reducer.fit_transform(embeddings)

print("✅ Dimensionality reduction complete!")

# Plot
plt.figure(figsize=(16, 10))

# Get unique clusters (excluding outliers for color assignment)
unique_clusters = sorted([c for c in df['hdbscan_topic_id'].unique() if c != -1])
n_clusters = len(unique_clusters)

# Create color map
colors = plt.cm.tab20(np.linspace(0, 1, min(n_clusters, 20)))
if n_clusters > 20:
    colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))

# Plot each cluster
for idx, cluster_id in enumerate(unique_clusters):
    mask = df['hdbscan_topic_id'] == cluster_id
    plt.scatter(
        embeddings_2d[mask, 0],
        embeddings_2d[mask, 1],
        c=[colors[idx % len(colors)]],
        label=f'Topic {cluster_id}' if idx < 20 else '',
        alpha=0.6,
        s=10
    )

# Plot outliers
outlier_mask = df['hdbscan_topic_id'] == -1
if outlier_mask.sum() > 0:
    plt.scatter(
        embeddings_2d[outlier_mask, 0],
        embeddings_2d[outlier_mask, 1],
        c='gray',
        label='Outliers',
        alpha=0.3,
        s=5
    )

plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.title(f'HDBSCAN Clusters Visualized with UMAP ({n_clusters} topics)')
if n_clusters <= 20:
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', markerscale=2)
plt.tight_layout()
plt.show()

print(f"\n✅ Visualization complete!")

## 9. Sample Topics

In [ ]:
# Show sample speeches from largest clusters
print("📋 Sample Speeches from Top 5 Largest Clusters:\n")

top_clusters = df['hdbscan_topic_id'].value_counts().head(6)
top_clusters = top_clusters[top_clusters.index != -1].head(5)

for cluster_id in top_clusters.index:
    cluster_df = df[df['hdbscan_topic_id'] == cluster_id]
    topic_label = topic_labels[cluster_id]
    
    print(f"\n{'='*80}")
    print(f"Topic {cluster_id}: {topic_label}")
    print(f"Size: {len(cluster_df):,} speeches")
    print(f"{'='*80}")
    
    # Show 3 sample speeches
    samples = cluster_df.sample(min(3, len(cluster_df)))
    
    for idx, (_, row) in enumerate(samples.iterrows(), 1):
        print(f"\n{idx}. Speech ID: {row['speech_id']}")
        print(f"   Speaker: {row.get('speech_giver', 'N/A')}")
        print(f"   Keywords: {row['keywords'][:100]}..." if len(str(row['keywords'])) > 100 else f"   Keywords: {row['keywords']}")

## 10. Save Results to CSV

In [ ]:
# Save updated dataframe with topic assignments
print(f"💾 Saving results to {OUTPUT_CSV}...")

df.to_csv(OUTPUT_CSV, index=False)

print(f"\n✅ Saved! CSV contains:")
print(f"   - speech_id")
print(f"   - keywords")
print(f"   - hdbscan_topic_id")
print(f"   - hdbscan_topic_label")
print(f"   - Other metadata columns")

print(f"\n📊 File info:")
print(f"   Rows: {len(df):,}")
print(f"   Columns: {len(df.columns)}")
print(f"   Size: {os.path.getsize(OUTPUT_CSV) / 1e6:.2f} MB")

# Display sample
print(f"\n📋 Sample output:")
df[['speech_id', 'hdbscan_topic_id', 'hdbscan_topic_label', 'keywords']].head(10)

## 11. Upload to Elasticsearch

In [ ]:
def upload_topics_to_elasticsearch(
    df: pd.DataFrame,
    es_host: str,
    es_index: str,
    batch_size: int = 100
) -> Tuple[int, int]:
    """
    Upload HDBSCAN topic assignments to Elasticsearch.
    
    Args:
        df: DataFrame with speech_id, hdbscan_topic_id, hdbscan_topic_label
        es_host: Elasticsearch host URL
        es_index: Index name
        batch_size: Number of documents to upload at once
    
    Returns:
        Tuple of (success_count, failure_count)
    """
    print(f"\n🔌 Connecting to Elasticsearch at {es_host}...")
    
    try:
        es = Elasticsearch(hosts=[es_host])
        
        if not es.ping():
            print("❌ Could not connect to Elasticsearch")
            return 0, 0
        
        print(f"✅ Connected to Elasticsearch")
        print(f"\n💾 Uploading {len(df):,} topic assignments...")
        
        # Prepare bulk update actions
        actions = []
        for _, row in df.iterrows():
            actions.append({
                '_op_type': 'update',
                '_index': es_index,
                '_id': row['speech_id'],
                'doc': {
                    'hdbscan_topic_id': int(row['hdbscan_topic_id']),
                    'hdbscan_topic_label': str(row['hdbscan_topic_label'])
                }
            })
        
        # Upload in batches
        success_count = 0
        failure_count = 0
        
        for i in tqdm(range(0, len(actions), batch_size), desc="Uploading"):
            batch = actions[i:i+batch_size]
            success, failed = helpers.bulk(es, batch, raise_on_error=False)
            success_count += success
            failure_count += len(failed) if failed else 0
        
        print(f"\n✅ Upload complete!")
        print(f"   Success: {success_count:,}")
        print(f"   Failures: {failure_count:,}")
        
        return success_count, failure_count
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return 0, 0

# Upload to Elasticsearch
success, failures = upload_topics_to_elasticsearch(
    df,
    ELASTICSEARCH_HOST,
    ELASTICSEARCH_INDEX,
    batch_size=100
)

## 12. Summary & Statistics

In [ ]:
print("="*80)
print("HDBSCAN CLUSTERING SUMMARY")
print("="*80)

n_clusters = len(df['hdbscan_topic_id'].unique()) - (1 if -1 in df['hdbscan_topic_id'].values else 0)
n_outliers = (df['hdbscan_topic_id'] == -1).sum()
n_clustered = len(df) - n_outliers

print(f"\n📊 Overall Statistics:")
print(f"   Total speeches: {len(df):,}")
print(f"   Number of topics: {n_clusters}")
print(f"   Clustered speeches: {n_clustered:,} ({n_clustered/len(df)*100:.1f}%)")
print(f"   Outliers: {n_outliers:,} ({n_outliers/len(df)*100:.1f}%)")

print(f"\n📁 Output Files:")
print(f"   Embeddings: {EMBEDDINGS_FILE}")
print(f"   Results CSV: {OUTPUT_CSV}")

print(f"\n🔍 Elasticsearch:")
print(f"   Successfully uploaded: {success:,} speeches")
print(f"   New fields added: hdbscan_topic_id, hdbscan_topic_label")

print(f"\n🎯 Top 10 Largest Topics:")
top_10 = df['hdbscan_topic_id'].value_counts().head(10)
for cluster_id, count in top_10.items():
    label = topic_labels.get(cluster_id, 'Unknown')
    if cluster_id == -1:
        print(f"   Topic {cluster_id:3d}: {count:6,} speeches - {label}")
    else:
        print(f"   Topic {cluster_id:3d}: {count:6,} speeches - {label[:60]}..." if len(label) > 60 else f"   Topic {cluster_id:3d}: {count:6,} speeches - {label}")

print("\n" + "="*80)
print("✅ CLUSTERING COMPLETE!")
print("="*80)